In [ ]:
# серверная
import socket
import time
import json
import os
import struct

PORT = 5555
FORMAT = 'utf-8'
HOST = 'localhost'

def get_process_info():
    process_list = []
    try:
        result = os.popen('ps -e -o pid,comm').readlines()
        for line in result[1:]:
            pid, name = line.strip().split(None, 1)
            process_list.append({"pid": int(pid), "name": name})
    except Exception as e:
        print(f"Ошибка при получении данных о процессах: {e}")
    return process_list

def save_to_xml(data, filename="process_info.xml"):
    xml_data = f'<?xml version="1.0" encoding="UTF-8"?>\n'
    xml_data += f'<process_info>\n'
    
    for process in data:
        xml_data += f'  <process>\n'
        for key, value in process.items():
            xml_data += f'    <{key}>{value}</{key}>\n'
        xml_data += f'  </process>\n'
    
    xml_data += f'</process_info>\n'
    
    # Сохраняем XML в файл
    with open(filename, 'w') as file:
        file.write(xml_data)
    
    print(f"Данные сохранены в {filename}")

def save_to_json(data, filename="process_info.json"):
    try:
        with open(filename, 'w') as file:
            json.dump(data, file, indent=4)
        print(f"Данные сохранены в {filename}")
    except Exception as e:
        print(f"Ошибка при сохранении в JSON: {e}")

def send_file(client_socket, filename="process_info.xml"):
    try:
        with open(filename, 'rb') as file:
            data = file.read()
            client_socket.sendall(data)
        print(f"Файл {filename} отправлен клиенту.")
    except Exception as e:
        print(f"Ошибка при отправке файла: {e}")

def send_signal_to_process(pid, signal_type):
    try:
        if signal_type == 'terminate':
            os.kill(pid, signal.SIGTERM)
            print(f"Отправлен сигнал завершения процессу с PID: {pid}")
        elif signal_type == 'kill':
            os.kill(pid, signal.SIGKILL)
            print(f"Отправлен сигнал принудительного завершения процессу с PID: {pid}")
        else:
            print(f"Неизвестный тип сигнала: {signal_type}")
    except Exception as e:
        print(f"Ошибка при отправке сигнала процессу {pid}: {e}")

def start_server(host=HOST, port=PORT):
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    while True:
        try:
            server_socket.bind((host, port))
            break
        except OSError:
            print(f"Порт {port} занят, пробую следующий порт...")
            port += 1

    server_socket.listen(5)
    print(f"Сервер запущен на {host}:{port}")
    
    while True:
        client_socket, client_address = server_socket.accept()
        print(f"Подключен клиент с адресом {client_address}")
        
        try:
            data_length = client_socket.recv(4)
            if not data_length:
                continue
            data_length = struct.unpack("I", data_length)[0]
            data = client_socket.recv(data_length).decode(FORMAT)
            
            # Преобразуем данные в команду
            command = json.loads(data)

            if command["action"] == "update":
                process_info = get_process_info()
                
                save_to_xml(process_info)
                save_to_json(process_info)
                
                send_file(client_socket)

            elif command["action"] == "signal":
                pid = command.get("pid")
                signal_type = command.get("signal_type", "terminate")
                send_signal_to_process(pid, signal_type)
            
            else:
                print("Неизвестная команда от клиента.")
        except Exception as e:
            print(f"Ошибка при обработке команды: {e}")
        finally:
            client_socket.close()

if __name__ == "__main__":
    start_server()

Сервер запущен на localhost:5555
Подключен клиент с адресом ('127.0.0.1', 52874)
Данные сохранены в process_info.xml
Данные сохранены в process_info.json
Файл process_info.xml отправлен клиенту.
Подключен клиент с адресом ('127.0.0.1', 52881)
Данные сохранены в process_info.xml
Данные сохранены в process_info.json
Файл process_info.xml отправлен клиенту.
